# Nathan: AI-Powered Research Optimizer for Quantum Computing

Nathan analyzes your quantum circuits against 1700+ papers from arXiv, D-Wave, Qiskit, PennyLane, and Classiq.
It provides optimization suggestions, hardware fit analysis, and relevant literature references.

**What Nathan does:**
1. Parses your circuit structure (qubits, gates, depth, pattern)
2. Classifies the problem type (QAOA, VQE, Grover, etc.)
3. Retrieves relevant papers from the scholar knowledge base
4. Generates optimization suggestions with QASM3 rewrites
5. Assesses hardware fit for specific backends

## Setup

Nathan is built into the `arvak` package. No extra install needed beyond `pip install arvak[nathan]`.

In [ ]:
import arvak

# Optional: configure API key for higher rate limits
# arvak.nathan.configure(api_key="nthn_...")
# Or set ARVAK_NATHAN_API_KEY environment variable

print(f"Arvak version: {arvak.__version__ if hasattr(arvak, '__version__') else 'dev'}")
print("Nathan module loaded:", hasattr(arvak, 'nathan'))

## Analyzing an Arvak Circuit

The simplest way to use Nathan — pass an `arvak.Circuit` directly.

In [ ]:
# Build a simple QAOA circuit
qaoa = arvak.Circuit("qaoa_maxcut", num_qubits=4, num_clbits=4)

# Initial superposition
for i in range(4):
    qaoa.h(i)

# Cost layer (ZZ interactions for graph edges)
for i, j in [(0,1), (1,2), (2,3), (0,3)]:
    qaoa.cx(i, j)
    qaoa.rz(0.7, j)  # gamma parameter
    qaoa.cx(i, j)

# Mixer layer
for i in range(4):
    qaoa.rx(1.0, i)  # 2*beta parameter

qaoa.measure_all()

print(f"Circuit: {qaoa.num_qubits} qubits, depth {qaoa.depth()}, {qaoa.size()} gates")
print(arvak.to_qasm(qaoa))

In [ ]:
# Analyze the circuit with Nathan
report = arvak.nathan.analyze(qaoa)

# In Jupyter, this renders as rich HTML automatically
report

In [ ]:
# Access individual report fields
print(f"Problem type: {report.problem_type}")
print(f"Quantum suitability: {report.suitability:.1%}")
print(f"Recommended algorithm: {report.recommended_algorithm}")
print(f"Estimated qubits: {report.estimated_qubits}")

if report.circuit:
    print(f"\nCircuit stats:")
    print(f"  Qubits: {report.circuit.num_qubits}")
    print(f"  Gates: {report.circuit.total_gates}")
    print(f"  Depth: {report.circuit.depth}")
    print(f"  Pattern: {report.circuit.detected_pattern}")

## Paper References

Nathan retrieves relevant papers from the scholar database (1700+ papers, 680+ method mappings).

In [ ]:
# Browse relevant papers
for i, paper in enumerate(report.papers, 1):
    print(f"{i}. {paper.title}")
    print(f"   {paper.arxiv_url}")
    if paper.relevance:
        print(f"   Relevance: {paper.relevance}")
    print()

## Optimization Suggestions

Nathan provides concrete QASM3 rewrites that you can apply directly.

In [ ]:
# Review suggestions
for s in report.suggestions:
    print(f"[{s.impact.upper() if s.impact else 'INFO'}] {s.title}")
    print(f"  {s.description}")
    if s.qasm3:
        print(f"  QASM3 rewrite available: {len(s.qasm3)} chars")
        # Convert suggestion to arvak.Circuit
        suggested_circuit = s.circuit
        if suggested_circuit:
            print(f"  -> {suggested_circuit.num_qubits} qubits, depth {suggested_circuit.depth()}")
    print()

## Analyzing Raw QASM3

You can pass QASM3 strings directly — no Arvak circuit needed.

In [ ]:
vqe_qasm = """
OPENQASM 3.0;
include "stdgates.inc";

// VQE ansatz for H2 molecule
qubit[4] q;
bit[4] c;

// Hartree-Fock initial state
x q[0];
x q[1];

// Single excitations
ry(0.1) q[2];
cx q[0], q[2];
ry(0.1) q[3];
cx q[1], q[3];

// Double excitation
cx q[0], q[1];
cx q[1], q[2];
cx q[2], q[3];
ry(0.05) q[3];
cx q[2], q[3];
cx q[1], q[2];
cx q[0], q[1];

c = measure q;
"""

vqe_report = arvak.nathan.analyze(vqe_qasm)
vqe_report

## Hardware-Specific Analysis

Pass a `backend` parameter to get hardware-specific recommendations.

In [ ]:
# Analyze for IQM Garnet
iqm_report = arvak.nathan.analyze(qaoa, backend="iqm_garnet")
print(f"Hardware fit: {iqm_report.hardware_fit}")
print(f"Recommended shots: {iqm_report.recommended_shots}")
print(f"Estimated error rate: {iqm_report.estimated_error_rate}")

## Framework Circuit Support

Nathan works with Qiskit, PennyLane, and Cirq circuits — auto-converted via existing Arvak integrations.

In [ ]:
# Qiskit example (requires pip install arvak[qiskit])
try:
    from qiskit import QuantumCircuit
    from qiskit.circuit import Parameter

    qc = QuantumCircuit(4, 4)
    gamma = Parameter('gamma')
    qc.h([0, 1, 2, 3])
    for i, j in [(0,1), (1,2), (2,3), (0,3)]:
        qc.cx(i, j)
        qc.rz(0.7, j)
        qc.cx(i, j)
    for i in range(4):
        qc.rx(1.0, i)
    qc.measure_all()

    # Nathan auto-converts Qiskit circuits
    qiskit_report = arvak.nathan.analyze(qc)
    print(f"Qiskit circuit analyzed: {qiskit_report.problem_type}")
    display(qiskit_report)

except ImportError:
    print("Qiskit not installed. Install with: pip install arvak[qiskit]")

## Chat with Nathan

Ask Nathan questions about quantum computing, algorithms, and hardware.

In [ ]:
response = arvak.nathan.chat(
    "What's the best algorithm for solving MaxCut on near-term hardware?"
)

# Rich HTML rendering in Jupyter
response

In [ ]:
# Follow-up question with context
response2 = arvak.nathan.chat(
    "How does the number of QAOA layers affect solution quality?",
    context=response.message
)
response2

## QAOA Improvement Workflow

A practical workflow: build circuit, analyze, apply suggestions, compare.

In [ ]:
# Step 1: Start with a basic QAOA circuit
original = arvak.Circuit("qaoa_v1", num_qubits=6, num_clbits=6)
for i in range(6):
    original.h(i)
# Naive all-to-all ZZ coupling
for i in range(6):
    for j in range(i+1, 6):
        original.cx(i, j)
        original.rz(0.5, j)
        original.cx(i, j)
for i in range(6):
    original.rx(1.0, i)
original.measure_all()

print(f"Original: {original.size()} gates, depth {original.depth()}")

# Step 2: Analyze with Nathan
analysis = arvak.nathan.analyze(original, backend="iqm_garnet")
print(f"\nProblem type: {analysis.problem_type}")
print(f"Suitability: {analysis.suitability:.1%}")
print(f"Suggestions: {len(analysis.suggestions)}")

# Step 3: Review suggestions
for s in analysis.suggestions:
    print(f"\n  [{s.impact}] {s.title}: {s.description}")

## Next Steps

- **Web interface:** Visit [arvak.io/nathan](https://arvak.io/nathan) for a browser-based editor
- **Dashboard:** Run `arvak dashboard` and navigate to the Nathan tab
- **API key:** Set `ARVAK_NATHAN_API_KEY` for higher rate limits (30 req/min free tier)
- **More notebooks:**
  - `01_core_arvak.ipynb` — Core circuit building API
  - `02_qiskit_integration.ipynb` — Qiskit framework support
  - `05_pennylane_integration.ipynb` — PennyLane QML workflows
  - `06_computational_chemistry.ipynb` — VQE for molecular simulation